# Projet 1

In [7]:
import re

import pandas as pd
import networkx as ntw
import numpy as np
import matplotlib.pyplot as plt

In [8]:
book =  open("LOTR_ReturnOfTheKing.txt").read()

book_chapters = re.split(r"_Chapter \w+_",book)[1:]



In [9]:
Characters = ["arwen", "aragorn", "bilbo", "boromir", "beregond", "bergil", "denethor", "elrond", "eomer", 
              "eowyn", "faramir", "frodo", "fredegard", "galadriel", "gandalf", "gimli",
              "gollum", "haldir", "isildur", "king of the deads", "legolas",
              "merry", "pippin", "sam", "saruman", "sauron", "theoden", "treebeard",
              "the witch king", "grima", "fatty", "rose cotton", "tom cotton", "shadowfax"]





In [10]:
## CLEANING 

cleaned_parag =[]
for ChapterNb, Chapter in enumerate(book_chapters):
    paragraphsEnsemble = Chapter.split('\n\n')
    for parag in paragraphsEnsemble :
        parag_new = re.sub(r"[!?',;.:']", ' ', parag)
        cleaned_parag.append(parag_new.lower())
        


Data transformation

In [12]:
CoocMatrix ={}
for parag in cleaned_parag: # Per paragraph, check if words are a Character

    List = []
    for w in parag.split():
        if w in Characters and w not in List:
            List.append(w)

    n = len(List)
    for i in range(n):
        for j in range(i+1,n):
            CoocMatrix[(List[i],List[j])] = CoocMatrix.get((List[i],List[j]) ,0)+1
            CoocMatrix[(List[j],List[i])] = CoocMatrix.get((List[j],List[i]) ,0)+1


print(CoocMatrix)

209
{('pippin', 'gandalf'): 43, ('gandalf', 'pippin'): 43, ('pippin', 'shadowfax'): 15, ('shadowfax', 'pippin'): 15, ('pippin', 'frodo'): 22, ('frodo', 'pippin'): 22, ('gandalf', 'shadowfax'): 16, ('shadowfax', 'gandalf'): 16, ('gandalf', 'frodo'): 23, ('frodo', 'gandalf'): 23, ('shadowfax', 'frodo'): 6, ('frodo', 'shadowfax'): 6, ('pippin', 'denethor'): 19, ('denethor', 'pippin'): 19, ('pippin', 'boromir'): 10, ('boromir', 'pippin'): 10, ('shadowfax', 'denethor'): 8, ('denethor', 'shadowfax'): 8, ('shadowfax', 'boromir'): 4, ('boromir', 'shadowfax'): 4, ('gandalf', 'denethor'): 22, ('denethor', 'gandalf'): 22, ('gandalf', 'boromir'): 11, ('boromir', 'gandalf'): 11, ('denethor', 'boromir'): 10, ('boromir', 'denethor'): 10, ('gandalf', 'aragorn'): 35, ('aragorn', 'gandalf'): 35, ('pippin', 'aragorn'): 21, ('aragorn', 'pippin'): 21, ('denethor', 'frodo'): 5, ('frodo', 'denethor'): 5, ('denethor', 'aragorn'): 10, ('aragorn', 'denethor'): 10, ('boromir', 'frodo'): 3, ('frodo', 'boromir'): 

In [13]:
# DataFrame Coocurence
df = pd.DataFrame(columns = Characters, index = Characters)
df[:] = int(0)

for key in CoocMatrix:
    if key[0]==key[1] :
        df[key[0]][key[1]] = CoocMatrix[key]
    else:        
        df[key[0]][key[1]] = CoocMatrix[key]



print(df)

                  arwen aragorn bilbo boromir beregond bergil denethor elrond  \
arwen                 0       5     1       0        0      0        0      5   
aragorn               5       0     4       5        5      2       10     20   
bilbo                 1       4     0       0        1      0        1      6   
boromir               0       5     0       0        5      0       10      0   
beregond              0       5     1       5        0      5       10      2   
bergil                0       2     0       0        5      0        1      2   
denethor              0      10     1      10       10      1        0      3   
elrond                5      20     6       0        2      2        3      0   
eomer                 0       0     0       0        0      0        0      0   
eowyn                 0       0     0       0        0      0        0      0   
faramir               2      15     1       7       10      2       14      4   
frodo                 3     